In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load data
data = fetch_california_housing()
X, y = data.data, data.target

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

#Model
class Ann_Regression(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(Ann_Regression, self).__init__()
        
        # Define layers
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        # Layer 1
        x = F.relu(self.layer1(x))
        # Layer 2
        x = F.relu(self.layer2(x))
        # Output layer
        x = self.layer3(x)
        return x

# Initialize network parameters
input_size = 8
hidden_size1 = 64
hidden_size2 = 32
output_size = 1
learning_rate = 0.001
epochs = 1000

# Initialize the network
net = Ann_Regression(input_size, hidden_size1, hidden_size2, output_size)

# Define optimizer and loss function
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
loss_cal = nn.MSELoss()

# Training loop
regression_train_loss = []
regression_val_loss = []
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(epochs):
    # Training mode
    net.train()
    
    # Zero gradients
    optimizer.zero_grad()
    
    # Forward pass
    predictions = net(X_train_tensor)
    
    # Calculate loss
    loss = loss_cal(predictions, y_train_tensor)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    # Validation phase
    net.eval()
    with torch.no_grad():
        val_predictions = net(X_test_tensor)
        val_loss = loss_cal(val_predictions, y_test_tensor)
    
    # Store losses
    regression_train_loss.append(loss.item())
    regression_val_loss.append(val_loss.item())
    
    # Early stopping check
    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break
    
    # Print progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, "
              f"Train Loss: {loss.item()}, "
              f"Val Loss: {val_loss.item()}")

# Evaluation
net.eval()
with torch.no_grad():
    test_predictions = net(X_test_tensor).cpu().numpy()
    mse = mean_squared_error(y_test, test_predictions)
    mae = mean_absolute_error(y_test, test_predictions)
    r2 = r2_score(y_test, test_predictions)
    
print(f"\nRegression Test Results:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")

# Plotting
plt.figure(figsize=(15, 5))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(regression_train_loss, label="Training Loss", color="blue")
plt.plot(regression_val_loss, label="Validation Loss", color="red")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)

# Plot predictions vs actual values
plt.subplot(1, 2, 2)
plt.scatter(y_test, test_predictions, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predictions vs Actual Values")
plt.grid(True)

plt.tight_layout()
plt.show()
